# Data Wrangling Using Pandas: Group Data

In [39]:
import dask.dataframe as dd
import numpy as np
%load_ext line_profiler
from scipy.stats import zscore

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:
# Load data sets <Shift-Tab> see inside function parenthesis to see more arguments.
df_bal = dd.read_csv('/Users/stewarta/Documents/DATA/Home Data/bureau_balance.csv')

In [3]:
df_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [4]:
df_bal = df_bal.set_index('SK_ID_BUREAU')

In [5]:
group_bal = df_bal.groupby('SK_ID_BUREAU')

In [26]:
# pure python: remove column access outside loop
def myapply1A(group):
    result = {}
    for name, g in group:
        x = g.values
        result[name] = (x - x.mean()) / x.std()
    return result

In [27]:
%prun -l 15 myapply1A(group_bal['MONTHS_BALANCE'])

TypeError: 'SeriesGroupBy' object is not iterable

In [31]:
## just apply the operation ???!!  result = df.groupby('id').max().reset_index().compute()
%prun -l 15 res = group_bal['MONTHS_BALANCE'].sum().compute()

         11904 function calls (11848 primitive calls) in 30.955 seconds

   Ordered by: internal time
   List reduced from 455 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      390   29.119    0.075   29.119    0.075 {method 'acquire' of '_thread.lock' objects}
       68    1.761    0.026    1.765    0.026 local.py:282(release_data)
       63    0.016    0.000   29.135    0.462 threading.py:263(wait)
      345    0.014    0.000    0.015    0.000 core.py:159(get_dependencies)
       69    0.007    0.000   29.144    0.422 queue.py:147(get)
        1    0.003    0.003    0.003    0.003 {built-in method posix.listdir}
        2    0.003    0.001    0.003    0.001 core.py:264(reverse_dict)
        1    0.002    0.002    0.007    0.007 order.py:83(order)
       69    0.002    0.000    0.013    0.000 local.py:469(fire_task)
      191    0.002    0.000    0.002    0.000 {built-in method builtins.sorted}
       69    0.002    0.000    1

In [43]:
%prun -l 15 res = group_bal['MONTHS_BALANCE'].apply(zscore).compute()

/Users/stewarta/anaconda3/envs/salliemae_env/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  if __name__ == '__main__':
/Users/stewarta/anaconda3/envs/salliemae_env/lib/python3.6/site-packages/scipy/stats/stats.py:2253: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


         17502 function calls (17291 primitive calls) in 197.600 seconds

   Ordered by: internal time
   List reduced from 727 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      371  194.875    0.525  194.875    0.525 {method 'acquire' of '_thread.lock' objects}
       62    2.465    0.040    2.487    0.040 local.py:282(release_data)
        2    0.065    0.032    0.065    0.032 {built-in method numpy.core.multiarray.concatenate}
        1    0.037    0.037  197.541  197.541 base.py:132(compute)
        2    0.011    0.005    0.011    0.005 {built-in method posix.rmdir}
        1    0.010    0.010  197.408  197.408 threaded.py:33(get)
        1    0.005    0.005  197.398  197.398 local.py:387(get_async)
      340    0.005    0.000    0.009    0.000 core.py:159(get_dependencies)
        1    0.005    0.005  197.599  197.599 <string>:1(<module>)
        2    0.004    0.002    0.005    0.002 {method 'readlines' of '_io._IOBase' ob

In [62]:
%prun -l 15 group_bal['MONTHS_BALANCE'].mean().compute() 

         23453 function calls (22954 primitive calls) in 32.524 seconds

   Ordered by: internal time
   List reduced from 756 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      396   30.216    0.076   30.216    0.076 {method 'acquire' of '_thread.lock' objects}
       69    2.228    0.032    2.235    0.032 local.py:282(release_data)
       64    0.009    0.000    0.009    0.000 threading.py:251(_acquire_restore)
        1    0.007    0.007    0.007    0.007 file.py:106(__del__)
        1    0.005    0.005   32.524   32.524 <string>:1(<module>)
      134    0.004    0.000    0.004    0.000 queue.py:202(_qsize)
       64    0.003    0.000   30.228    0.472 threading.py:263(wait)
      350    0.003    0.000    0.004    0.000 core.py:159(get_dependencies)
       70    0.002    0.000    2.238    0.032 local.py:298(finish_task)
        1    0.002    0.002    0.002    0.002 order.py:169(ndependents)
       18    0.001    0.000    0.00

In [89]:
%prun -l 15 means = group_bal['MONTHS_BALANCE'].apply(lambda x: (z = x.mean()) / x.std(); x - z).compute()

/Users/stewarta/anaconda3/envs/salliemae_env/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  if __name__ == '__main__':


         20259 function calls (20013 primitive calls) in 696.245 seconds

   Ordered by: internal time
   List reduced from 790 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      354  693.858    1.960  693.858    1.960 {method 'acquire' of '_thread.lock' objects}
       62    1.692    0.027    1.719    0.028 local.py:282(release_data)
        4    0.429    0.107    0.429    0.107 {built-in method numpy.core.multiarray.concatenate}
        1    0.109    0.109  696.176  696.176 base.py:132(compute)
       54    0.024    0.000  693.882   12.850 threading.py:263(wait)
      340    0.014    0.000    0.015    0.000 core.py:159(get_dependencies)
        1    0.013    0.013    0.013    0.013 {built-in method posix.listdir}
        1    0.006    0.006    0.026    0.026 file.py:106(__del__)
        2    0.005    0.002    0.005    0.002 {built-in method posix.rmdir}
        1    0.004    0.004    0.065    0.065 groupby.py:1049(apply)
     

In [90]:
# pure python: remove column access outside loop
def myapply1B(group):
    result = {}
    for name, g in group:
        x = g.values
        m = x.mean()
        std = x.std()
        result[name] = (x - m ) / std
    return result

In [91]:
%prun -l 15 myapply1B(group_bal['MONTHS_BALANCE'])

TypeError: 'SeriesGroupBy' object is not iterable